# BioLab API Demo (BioLab container)

In [1]:
# TODO: create sep repo and pypi for data model so that you can install it and import like `import sed_data as sed`

In [2]:
import sys
import os

sys.path.insert(0, '..')

#### **_Experiment 1_**:  Here we cross a boundary in the stack that is a biological simulation. We go from model configuration, to experiment. Thus, this tooling sits at that level: both experiment specification AND experiment execution. It's not just a way to specify an experiment, but it is also a way to run it, given the many knobs and buttons that you can use predefine and customize the way the actual model is being solved. Our users are seeking to be involved in an experiment as a "stack". We should make a bigger distinction between terms like "model".

In [3]:
from process_bigraph import pp
from biosimulator_processes.biosimulator_builder import BuildPrompter
from biosimulator_processes.data_model.sed_data_model import SedDataModel as sed 

In [4]:
# 1a. define a model for the process composition. In this case, just one model to be re-used as configuration for the processes we create:
tumor_control_biomodel_id = 'BIOMD0000000749'
simple_tc_model = sed.TimeCourseModel(model_source=tumor_control_biomodel_id)

pp(simple_tc_model)

In [5]:
# 1b. view model info for model source:

pp(simple_tc_model.source_info)

In [6]:
# 1c. define a TimeCourse process instance using the above object as a parameter. The other parameter is method. See BasiCO documentation for more details on solvers

simple_tc_process = sed.TimeCourseProcess(model=simple_tc_model, method='lsoda')

In [7]:
# >> The process model instance is viewable as a dataclass...

pp(simple_tc_process)

In [8]:
# >> ...or a dict:

pp(simple_tc_process.to_dict())

In [9]:
# 2. instantiate the prompter:

prompter = BuildPrompter()

In [10]:
# 3. add process(es) to the bigraph with the Time Course model instance we created above. For now, just one process will be added.

prompter.add_single_process(config=simple_tc_process)

In [ ]:
# 4. Inspect the builder instance within prompter:

pp(prompter.builder_instance)

In [ ]:
# 5. Visualize the fully-connected composition:

prompter.visualize_bigraph()

In [ ]:
# 6. Generate a composite engine and use to execute the bigraph that we just created:

prompter.run()

In [ ]:
# 7. Clear the builder and start from scratch, writing it out first:

prompter.flush(fp='simple_single_demo')

In [ ]:
prompter.builder_instance

#### **_Experiment 2_**: Load an SBML model from a specified model filepath and add Model changes to the composite before adding it to the bigraph. Here, we expect the user to be familiar enough with the model file they are passing to make individual species/parameter/reaction changes for specific species types. In the Caravagna model, for example, the species involved are T, E, I. Let's change the initial concentration for some of these as an example of model changes:

In [ ]:
caravagna_model_filepath = '../biosimulator_processes/model_files/Caravagna2010.xml'
print(os.path.exists(caravagna_model_filepath))

In [ ]:
# first make the timecourse model which is easily configured with objects related to model changes
adjusted_tc_model_from_file = sed.TimeCourseModel(
    model_source=caravagna_model_filepath, 
    model_changes=sed.TimeCourseModelChanges(
        species_changes=sed.SpeciesChange(species_name='T', initial_concentration=0.234)
    )
)


pp(adjusted_tc_model_from_file)

In [ ]:
adjusted_tc_process = sed.TimeCourseProcess(model=adjusted_tc_model_from_file, method='stochastic')

pp(adjusted_tc_process)

In [ ]:
# add the process we just created with the prompter. The prompter will create a new instance of builder for us, if not passed:

prompter.add_single_process(config=adjusted_tc_process)

In [ ]:
# visualize the adjusted creation

prompter.visualize_bigraph()

In [ ]:
# view the builder document

prompter.builder_instance.document()

In [ ]:
# run the composite

prompter.run()

In [ ]:
# stochastic works with Caravagna!!